<h1>Test set path</h1>

In [1]:
test_set = "C:/Users/Theo/Documents/Unif/detection test set" #insert path

<h3>Generic function to compute intersection over union.</h3>

In [2]:
def yolo_to_relative_coord(bbox, img_dim):
    x_center, y_center, width, height = bbox
    img_w, img_h = img_dim
    
    x_min = (x_center - width / 2) * img_w
    y_min = (y_center - height / 2) * img_h
    x_max = (x_center + width / 2) * img_w
    y_max = (y_center + height / 2) * img_h
    
    return [x_min, y_min, x_max, y_max]

In [3]:
"""
input:
bbox1, bbox2: bounding boxes in YOLO format
YOLO format ==> [x_center, y_center, width, height], normailzed coordinates

output:
The intersection over union metric between bbox1 and bbox2
"""
def iou(bbox1, bbox2, img_dim=(1080, 1920)):

    # conversion into relative coordinates
    Ax, Ay, Bx, By = yolo_to_relative_coord(bbox1, img_dim)
    Cx, Cy, Dx, Dy = yolo_to_relative_coord(bbox2, img_dim)

    # computation of the intersection
    x_overlap = min(Dx, Bx) - max(Ax, Cx)
    y_overlap = min(Dy, By) - max(Ay, Cy)
    if (x_overlap < 0 or y_overlap < 0): return 0 # no overlap case
    intersection = x_overlap*y_overlap

    # computation of the union
    area_1 = abs((Bx-Ax)*(By-Ay))
    area_2 = abs((Dx-Cx)*(Dy-Cy))
    union = area_1 + area_2 - intersection

    # IoU
    return intersection/union


<h3>Test of the <i>iou</i> function.</h3>

In [4]:
def test(observed, expected, test_name=""):
    msg = f'({test_name}): Observed: {observed}; Expected: {expected}'
    if (round(observed, 5) == round(expected, 5)): print(f"Test passed ({test_name})")
    else: print(f"Test failed{msg}")

In [5]:
bbox = [0.1, 0.1, 0.2, 0.2]
test(iou(bbox, bbox), 1, "Perfect Overlap") # perfect overlap

bbox2 = [0.9, 0.9, 0.1, 0.1] 
test(iou(bbox, bbox2), 0, "No Overlap") # no overlap

bbox3 = [0.1, 0.2, 0.2, 0.2] 
test(iou(bbox, bbox3), 1/3, "No Overlap") # no overlap

Test passed (Perfect Overlap)
Test passed (No Overlap)
Test passed (No Overlap)


<h2>Computation of the following metrics based on the ground truth (from the test set) and the predictions</h2>
<h3>
    <ul>
        <li>True Positive</li>
        <li>False Positive</li>
        <li><span style="color:grey;">True Negative: not counted (see below)</span></li>
        <li>False Negative</li>
    </ul> 
</h3>

In [6]:
"""
The arguments <ground_truths> and <predictions> are expected to be 
python dictionaries structured as follows:

    dictionnary {
        image_name 1: [
                bbox 1,
                bbox 2, 
                ...
            ],
        image_name 2: [
                bbox 1,
                bbox 2, 
                ...
            ],
        ...
    }

PS: the bounding boxes are expected to be encoded in YOLO format.

    
For each image, every prediction bbox will be 
compared to every ground truth bbox to find the best match.

If a ground truth bbox finds no match among the prediction bbox:
+1 false negative

If a prediction bbox finds no match among the ground truth bbox:
+1 false positive

If the best match between a prediction bbox and the ground truth
bboxes has a low IoU (bellow a given threshold <t>):
+1 false positive

If a prediction bbox best match in the ground truth bboxes has
already been matched by another prediction bbox with a greater score:
+1 false positive

If the best match between a prediction bbox and the ground truth
bboxes has a high IoU (above a given threshold <t>):
+1 true positive

The true negative won't be counted because it means that no
prediction bbox has been found in the background. This is nonsense to count this.
"""
def extract_metrics(ground_truths:dict, predictions:dict, t=0.75):
    tp, fp, fn = 0, 0, 0  # Initialize counters for TP, FP, and FN
    
    # Iterate over all unique image names in ground truths and predictions
    for image_name in set(ground_truths.keys()).union(predictions.keys()):
        gt_bboxes = ground_truths.get(image_name, [])  # Retrieve ground truth bboxes (default empty list if missing)
        pred_bboxes = predictions.get(image_name, [])  # Retrieve predicted bboxes (default empty list if missing)
        
        matched_gt = set()  # Store indices of matched ground truth bboxes
        pred_matched_scores = []  # Keep track of IoU scores of matched predictions
        
        # Iterate over each predicted bounding box
        for pred in pred_bboxes:
            best_iou = 0  # Initialize the best IoU score for the current prediction
            best_gt_idx = -1  # Index of the best-matching ground truth bbox
            
            # Compare prediction with each ground truth bbox
            for i, gt in enumerate(gt_bboxes):
                score = iou(pred, gt)  # Compute IoU
                if score > best_iou:  # Update best match if IoU is higher
                    best_iou = score
                    best_gt_idx = i
            
            # Determine if the prediction is a TP or FP based on IoU and previous matches
            if best_iou >= t and best_gt_idx not in matched_gt:
                matched_gt.add(best_gt_idx)  # Mark ground truth bbox as matched
                pred_matched_scores.append(best_iou)  # Store IoU score
                tp += 1
            else:
                fp += 1
        
        # Count False Negatives (ground truths that were not matched)
        fn += len(gt_bboxes) - len(matched_gt)
    
    return {"true_positives": tp, "false_positives": fp, "false_negatives": fn}


<h3>Test of the <i>extract_metrics</i> function.</h3>

In [7]:
print("\n------------------------------------")
# perfect match
GT = {"img1": [[0.2, 0.2, 0.1, 0.1]]}
PRED = {"img1": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "perfect match (TP)")
test(fp, 0, "perfect match (FP)")
test(fn, 0, "perfect match (FN)")

print("\n------------------------------------")
# No overlap
GT = {"img2": [[0.8, 0.8, 0.1, 0.1]]}
PRED = {"img2": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 0, "No overlap (TP)")
test(fp, 1, "No overlap (FP)")
test(fn, 1, "No overlap (FN)")

print("\n------------------------------------")
# No overlap but bboxes are closer
GT = {"img3": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img3": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 0, "No overlap but bboxes are closer (TP)")
test(fp, 1, "No overlap but bboxes are closer (FP)")
test(fn, 1, "No overlap but bboxes are closer (FN)")

print("\n------------------------------------")
# Not a perfect match but above threshold
GT = {"img4": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img4": [[0.1, 0.1, 0.09, 0.09]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Not a perfect match but above threshold (TP)")
test(fp, 0, "Not a perfect match but above threshold (FP)")
test(fn, 0, "Not a perfect match but above threshold (FN)")

print("\n------------------------------------")
# Two prediction bboxes matching the same ground truth
GT = {"img5": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 1, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 0, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# Composed case - 2 pred matching 1 GT - 1 pred matching 1 GT
GT = {"img6": [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]]}
PRED = {"img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 2, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 1, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 0, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# One prediction overlapping two ground truths
GT = {"img7": [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.346540]]}
PRED = {"img7": [[0.367304, 0.808140, 0.362103, 0.366419]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 0, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 1, "Two prediction bboxes matching the same ground truth (FN)")



------------------------------------
Test passed (perfect match (TP))
Test passed (perfect match (FP))
Test passed (perfect match (FN))

------------------------------------
Test passed (No overlap (TP))
Test passed (No overlap (FP))
Test passed (No overlap (FN))

------------------------------------
Test passed (No overlap but bboxes are closer (TP))
Test passed (No overlap but bboxes are closer (FP))
Test passed (No overlap but bboxes are closer (FN))

------------------------------------
Test passed (Not a perfect match but above threshold (TP))
Test passed (Not a perfect match but above threshold (FP))
Test passed (Not a perfect match but above threshold (FN))

------------------------------------
Test passed (Two prediction bboxes matching the same ground truth (TP))
Test passed (Two prediction bboxes matching the same ground truth (FP))
Test passed (Two prediction bboxes matching the same ground truth (FN))

------------------------------------
Test passed (Two prediction bboxes

<h3>Combined test of the <i>extract_metrics</i> function.</h3>

In [8]:
GT = {
    "img1": [[0.2, 0.2, 0.1, 0.1]], 
    "img2": [[0.8, 0.8, 0.1, 0.1]], 
    "img3": [[0.1, 0.1, 0.1, 0.1]],
    "img4": [[0.1, 0.1, 0.1, 0.1]],
    "img5": [[0.1, 0.1, 0.1, 0.1]],
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]],
    "img7": [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.346540]]
    }
PRED = {
    "img1": [[0.2, 0.2, 0.1, 0.1]],    # +1 TP
    "img2": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img3": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img4": [[0.1, 0.1, 0.09, 0.09]],  # +1 TP --> not a perfect match but above threshold
    "img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]], # +1 TP, +1 FP
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]], # +2 TP, +1 FP
    "img7": [[0.367304, 0.808140, 0.362103, 0.366419]] # +1 TP, +1 FN
    }

tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 6, "Combined test (TP)")
test(fp, 4, "Combined test (FP)")
test(fn, 3, "Combined test (FN)")

Test passed (Combined test (TP))
Test passed (Combined test (FP))
Test passed (Combined test (FN))


<h2>Extract the data from the test set (to <i>dict</i>)</h2>

In [23]:
import os

def extract_ground_truth(test_set_path = test_set):
    path = f'{test_set_path}/labels/obj_train_data'
    data = dict()

    # check whether the specified path is valid directory
    if not os.path.isdir(path):
        print(f"Error: {path} is not a valid directory.")
        return
    
    for textfile in os.listdir(path):
        file_path = f"{path}/{textfile}"

        # check whether textfile exists in the folder located at the specified path
        if os.path.isfile(file_path):
            data[textfile.strip(".txt")] = []

            # open the file in read mode
            with open(file_path, 'r') as file:

                # extract the bounding boxes one by one
                for line in file.readlines():
                    splitted = line.split(" ")
                    detection_class = splitted[0]
                    if (detection_class == 0): continue # abort iteration if the class is a face
                    bbox = splitted[1:]
                    for i in range(len(bbox)): bbox[i] = float(bbox[i])
                    data[textfile.strip(".txt")].append(bbox)
                file.close()
    
    return data


<h3>Test of the <i>extract_ground_truth</i> function.</h3>

In [18]:
GT = extract_ground_truth(test_set_path="test_set_example")
PRED = {
    "img1": [[0.2, 0.2, 0.1, 0.1]],    # +1 TP
    "img2": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img3": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img4": [[0.1, 0.1, 0.09, 0.09]],  # +1 TP --> not a perfect match but above threshold
    "img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]], # +1 TP, +1 FP
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]], # +2 TP, +1 FP
    "img7": [[0.367304, 0.808140, 0.362103, 0.366419]] # +1 TP, +1 FN
    }
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 6, "Extract ground truth from files (TP)")
test(fp, 4, "Extract ground truth from files (FP)")
test(fn, 3, "Extract ground truth from files (FN)")

print(GT)

Test passed (Extract ground truth from files (TP))
Test passed (Extract ground truth from files (FP))
Test passed (Extract ground truth from files (FN))
{'img1': [[0.2, 0.2, 0.1, 0.1]], 'img2': [[0.8, 0.8, 0.1, 0.1]], 'img3': [[0.1, 0.1, 0.1, 0.1]], 'img4': [[0.1, 0.1, 0.1, 0.1]], 'img5': [[0.1, 0.1, 0.1, 0.1]], 'img6': [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]], 'img7': [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.34654]]}


<h1>Effective code</h1>

<h3>Ground truth extraction.</h3>

In [24]:
GT = extract_ground_truth()

<h3>Predictions extraction</h3>

In [25]:
print(GT)

{'20241015 - 11h46_frame_11231': [[0.47275, 0.309079, 0.129031, 0.249102], [0.432786, 0.705769, 0.510958, 0.588463], [0.238435, 0.563574, 0.122203, 0.299519], [0.282727, 0.311769, 0.113911, 0.19713]], '20241015 - 11h46_frame_12495': [[0.58418, 0.57444, 0.500995, 0.793269], [0.727607, 0.943042, 0.295818, 0.113917], [0.880487, 0.820866, 0.239026, 0.358269], [0.644586, 0.350162, 0.131359, 0.319954]], '20241015 - 11h46_frame_13051': [[0.630992, 0.37081, 0.056776, 0.081454], [0.332685, 0.034898, 0.069724, 0.069796], [0.535169, 0.49837, 0.037484, 0.081352], [0.335674, 0.813782, 0.141432, 0.372435], [0.516948, 0.593032, 0.111552, 0.270917]], '20241015 - 11h46_frame_13182': [[0.507555, 0.248481, 0.019161, 0.028704], [0.51944, 0.25306, 0.058766, 0.054898], [0.347625, 0.452264, 0.177292, 0.254972]], '20241015 - 11h46_frame_14122': [[0.366052, 0.326542, 0.43924, 0.53475], [0.213661, 0.10838, 0.166333, 0.216759], [0.390409, 0.141782, 0.090984, 0.16175], [0.882479, 0.033125, 0.235042, 0.06625], [0.